In [1]:
import pandas as pd
import os

In [2]:
import tensorflow as tf

print(f"tf.__version__ = {tf.__version__}")

tf.__version__ = 2.13.0


In [3]:
data_path = "../data/KuaiRec/data/"
os.listdir(data_path)

['item_daily_features.csv',
 'item_categories.csv',
 'user_features.csv',
 'small_matrix.csv',
 'social_network.csv',
 'big_matrix.csv']

In [4]:
small_matrix = pd.read_csv(data_path + "small_matrix.csv")
item_daily_features = pd.read_csv(data_path + "item_daily_features.csv")
user_features = pd.read_csv(data_path + "user_features.csv")

In [5]:
print(f"small_matrix.shape = {small_matrix.shape}")
small_matrix.head()

small_matrix.shape = (4676570, 8)


,user_id,video_id,play_duration,video_duration,time,date,timestamp,watch_ratio
0,14,148,4381,6067,2020-07-05 05:27:48.378,20200705.0,1.593898e+09,0.722103
1,14,183,11635,6100,2020-07-05 05:28:00.057,20200705.0,1.593898e+09,1.907377
2,14,3649,22422,10867,2020-07-05 05:29:09.479,20200705.0,1.593898e+09,2.063311
3,14,5262,4479,7908,2020-07-05 05:30:43.285,20200705.0,1.593898e+09,0.566388
4,14,8234,4602,11000,2020-07-05 05:35:43.459,20200705.0,1.593899e+09,0.418364


In [6]:
small_matrix = small_matrix[["user_id", "video_id", "date", "watch_ratio"]]

In [7]:
print(f"item_daily_features.shape={item_daily_features.shape}")
item_daily_features.head()

item_daily_features.shape=(343341, 58)


,video_id,date,author_id,video_type,upload_dt,upload_type,visible_status,video_duration,video_width,video_height,...,download_cnt,download_user_num,report_cnt,report_user_num,reduce_similar_cnt,reduce_similar_user_num,collect_cnt,collect_user_num,cancel_collect_cnt,cancel_collect_user_num
0,0,20200705,3309,NORMAL,2020-03-30,ShortImport,public,5966.0,720,1280,...,8,8,0,0,3,3,NaN,NaN,NaN,NaN
1,0,20200706,3309,NORMAL,2020-03-30,ShortImport,public,5966.0,720,1280,...,2,2,0,0,5,5,NaN,NaN,NaN,NaN
2,0,20200707,3309,NORMAL,2020-03-30,ShortImport,public,5966.0,720,1280,...,2,2,0,0,0,0,NaN,NaN,NaN,NaN
3,0,20200708,3309,NORMAL,2020-03-30,ShortImport,public,5966.0,720,1280,...,3,3,0,0,3,3,NaN,NaN,NaN,NaN
4,0,20200709,3309,NORMAL,2020-03-30,ShortImport,public,5966.0,720,1280,...,2,2,2,1,1,1,NaN,NaN,NaN,NaN


In [8]:
print(f"user_features.shape={user_features.shape}")
user_features.head()

user_features.shape=(7176, 31)


,user_id,user_active_degree,is_lowactive_period,is_live_streamer,is_video_author,follow_user_num,follow_user_num_range,fans_user_num,fans_user_num_range,friend_user_num,...,onehot_feat8,onehot_feat9,onehot_feat10,onehot_feat11,onehot_feat12,onehot_feat13,onehot_feat14,onehot_feat15,onehot_feat16,onehot_feat17
0,0,high_active,0,0,0,5,"(0,10]",0,0,0,...,184,6,3,0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,full_active,0,0,0,386,"(250,500]",4,"[1,10)",2,...,186,6,2,0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,full_active,0,0,0,27,"(10,50]",0,0,0,...,51,2,3,0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,full_active,0,0,0,16,"(10,50]",0,0,0,...,251,3,2,0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,full_active,0,0,0,122,"(100,150]",4,"[1,10)",0,...,99,4,2,0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
interaction_with_item_features = pd.merge(left=small_matrix, right=item_daily_features, how='left', on=['video_id', 'date'])
dataset_full = pd.merge(left=interaction_with_item_features, right=user_features, how='left', on='user_id')

In [10]:
train_data = dataset_full[dataset_full.date < 20200820].drop(columns=['date']).dropna().sample(frac=0.1)

In [11]:
print(f"train_data.shape={train_data.shape}")
train_data.head()

train_data.shape=(171570, 89)


,user_id,video_id,watch_ratio,author_id,video_type,upload_dt,upload_type,visible_status,video_duration,video_width,...,onehot_feat8,onehot_feat9,onehot_feat10,onehot_feat11,onehot_feat12,onehot_feat13,onehot_feat14,onehot_feat15,onehot_feat16,onehot_feat17
3257219,5029,9117,0.787906,7837.0,NORMAL,2020-08-12,Web,public,7700.0,720.0,...,284,6,3,0,0.0,0.0,0.0,1.0,0.0,0.0
4578268,7044,8457,0.914375,5097.0,NORMAL,2020-07-11,ShortImport,public,11200.0,540.0,...,92,4,3,0,0.0,0.0,0.0,0.0,0.0,0.0
4155129,6330,6235,1.319261,895.0,NORMAL,2020-08-12,ShortCamera,public,9300.0,720.0,...,152,3,2,0,0.0,0.0,0.0,0.0,0.0,0.0
2709465,4306,7127,0.021846,2293.0,NORMAL,2020-07-17,ShortImport,public,9383.0,720.0,...,174,6,2,0,1.0,0.0,0.0,0.0,0.0,0.0
601338,1010,8628,1.660474,748.0,NORMAL,2020-07-25,ShortImport,public,5866.0,719.0,...,272,4,2,0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
item_continual_features = [
    'video_duration',
    'show_cnt',
    'show_user_num',
    'play_cnt',
    'play_user_num',
]
item_int_cate_features = [
    'video_id',
]
item_string_cate_features = [
    'video_type'
]
user_continual_features = [
    'fans_user_num',
    'friend_user_num',
    'register_days',
]
user_int_cate_features = [
    'user_id',
    'is_lowactive_period',
    'is_live_streamer',
    'is_video_author',
]
user_string_cate_features = [
    'user_active_degree',
    'follow_user_num_range',
    'fans_user_num_range',
    'friend_user_num_range',
    'register_days_range',

]
user_features = user_continual_features + user_int_cate_features + user_string_cate_features
item_features = item_continual_features + item_int_cate_features + item_string_cate_features
label = 'watch_ratio'

In [13]:
# tf model fit에서 사용할 데이터
tf_train_data = tf.data.Dataset.from_tensor_slices(dict(train_data[user_features+item_features+[label]]))

In [14]:
# tfrs에서 metrics 계산시 사용
items = tf.data.Dataset.from_tensor_slices(dict(train_data[item_features]))

In [15]:
items.element_spec

{'video_duration': TensorSpec(shape=(), dtype=tf.float64, name=None),
 'show_cnt': TensorSpec(shape=(), dtype=tf.float64, name=None),
 'show_user_num': TensorSpec(shape=(), dtype=tf.float64, name=None),
 'play_cnt': TensorSpec(shape=(), dtype=tf.float64, name=None),
 'play_user_num': TensorSpec(shape=(), dtype=tf.float64, name=None),
 'video_id': TensorSpec(shape=(), dtype=tf.int64, name=None),
 'video_type': TensorSpec(shape=(), dtype=tf.string, name=None)}

In [16]:
# embedding layer를 위해
item_int_vocabs = {x: train_data[x].unique() for x in item_int_cate_features}
item_str_vocabs = {x: train_data[x].unique() for x in item_string_cate_features}

user_int_vocabs = {x: train_data[x].unique() for x in user_int_cate_features}
user_str_vocabs = {x: train_data[x].unique() for x in user_string_cate_features}

In [17]:
# normalize를 위해
item_statistics = {x: (train_data[x].mean(), train_data[x].var()) for x in item_continual_features}
user_statistics = {x: (train_data[x].mean(), train_data[x].var()) for x in user_continual_features}

# Two-Tower

In [18]:
import sys
from pathlib import Path

current_dir = Path(Path.cwd()).resolve()
parent_current_dir = str(current_dir.parent)
print(f"{parent_current_dir} is add!")
sys.path.append(parent_current_dir)

/Users/kakao/study/RecModel is add!


In [19]:
from src.model.TwoTower_tf import TwoTower, TowerModel, RepresentLayer

In [20]:
print(f"user embedding 크기 {len(user_int_cate_features) * 8 + len(user_string_cate_features) * 8 + len(user_continual_features)}")
print(f"user embedding 크기 {len(item_int_cate_features) * 8 + len(item_string_cate_features) * 8 + len(item_continual_features)}")

user embedding 크기 75
user embedding 크기 21


In [21]:
two_tower = TwoTower(
        user_int_vocabs=user_int_vocabs,
        user_str_vocabs=user_str_vocabs,
        user_statistics=user_statistics,
        user_features=user_features,
        user_dense_layer_size_list=[128, 64, 32],
        item_int_vocabs=item_int_vocabs,
        item_str_vocabs=item_str_vocabs,
        item_statistics=item_statistics,
        item_features=item_features,
        item_dense_layer_size_list=[32],
        tf_dataset_item=items,
    )

In [22]:
# WARNING:At this time, the v2.11+ optimizer `tf.keras.optimizers.Adam` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.Adam`.
# WARNING:There is a known slowdown when using v2.11+ Keras optimizers on M1/M2 Macs. Falling back to the legacy Keras optimizer, i.e., `tf.keras.optimizers.legacy.Adam`.
two_tower.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.01))

In [23]:
cached_train = (
    tf_train_data
        .filter(lambda x: x['watch_ratio'] >= 1.0) # positive의 정의
        .map(lambda x: {
            key: tf.cast(value, tf.float32)
            if key in item_continual_features + user_continual_features + [label] else value
            for key, value in x.items()
    })
    .shuffle(10000).batch(512).cache()
)

In [27]:
two_tower_history = two_tower.fit(cached_train, epochs=5, verbose=1)

Epoch 1/5
111/111 [==============================] - 1s 6ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 6.2299 - regularization_loss: 0.0000e+00 - total_loss: 6.2299
Epoch 2/5
111/111 [==============================] - 1s 6ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 6.2288 - regularization_loss: 0.0000e+00 - total_loss: 6.2288
Epoch 3/5
111/111 [==============================] - 1s 6ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/